# Exploring HGCal

Let's take a look at some hgcal data.  First some simple visualizations...

In [11]:
#imports and setup
%matplotlib inline
%cd /home/naodell/work/CMS/hgcal/hgcal_analysis

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import plotly.plotly as plotly
import plotly.graph_objs as go
from plotly.graph_objs import Surface, Mesh3d
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
    
from scripts.helpers import eta_to_theta, propagate_to_face, assign_phi

sys.path.append('/usr/local/lib')
from root_pandas import read_root

/home/naodell/work/CMS/hgcal/hgcal_analysis


In [ ]:
# get the rechits and apply some cuts
event_data = pd.read_csv('data/pions/rechits.csv', names=['evt', 'x', 'y', 'z', 't', 'e'])
event_data = event_data.query('z > 0. and evt == 3')

# add some additional vars
event_data['r'] = np.sqrt(event_data.x**2 + event_data.y**2)
event_data['phi'] = np.arctan(np.abs(event_data.y/event_data.x)) 
assign_phi(event_data) # needed to correctly map phi onto 0 to 2pi

# get the gen particles 
gen_data = pd.read_csv('data/pions/genparticles.csv', names=['evt', 'pdgid', 'eta', 'phi', 'pt', 'e'])
gen_data['theta'] = 2*np.arctan(np.exp(-gen_data.eta))
gen_data = gen_data.query('theta > 0. and evt == 3')

In [ ]:
#plt.yscale('log')
fig, axes = plt.subplots(1, 2, figsize=(12,6), sharey=True, facecolor='white')
axes[0].hist(event_data.e, range=(0., 2.), bins=50, histtype='step', lw=2)
axes[0].set_xlabel('rechit energy')
axes[0].set_ylabel('hits')
axes[0].set_title('')
axes[0].set_xlim([0., 2.])

axes[1].hist(event_data.t, range=(0.99, 1.15), bins=20, histtype='step', lw=2)
axes[1].set_xlabel('rechit time')
axes[1].set_ylabel('hits')
axes[1].set_title('')
axes[1].set_xlim([0.99, 1.15])

plt.show()

In [ ]:
scan_data = event_data#.query('sqrt((x-20)**2 + (y - 60)**2) < 20. and 0. < phi < {0}/2'.format(np.pi))
phi_cscale = scan_data.phi/(2*np.pi)

gen_face_position = np.array([propagate_to_face(p.theta, p.phi, p.pt, 3.2, 0.14) for i,p in gen_data.iterrows()])
gen_tail_position = np.array([propagate_to_face(p.theta, p.phi, p.pt, 4.1, 0.14) for i,p in gen_data.iterrows()])

fig, axes = plt.subplots(1, 2, figsize=(16,8), facecolor='white')
axes[0].scatter(scan_data.z, np.sign(scan_data.y)*scan_data.r, s=20*scan_data.e, c=phi_cscale, cmap='viridis')
axes[0].set_xlabel('z (cm)')
axes[0].set_ylabel('r (cm)')
axes[0].set_xlim([315., 420.])
axes[0].set_ylim([50., 80.])

axes[1].scatter(100.*gen_face_position[:,0], 100.*gen_face_position[:,1], marker='*', c='k')
axes[1].scatter(100.*gen_tail_position[:,0], 100.*gen_tail_position[:,1], marker='>', c='k')
axes[1].hist2d(scan_data.x, scan_data.y, bins=100, cmap='hot_r', range=[[-180, 180], [-180, 180]])
axes[1].set_xlabel('x (cm)')
axes[1].set_ylabel('y (cm)')
axes[1].set_xlim([0., 175.])
axes[1].set_ylim([0., 175.])

plt.show()

In [ ]:
#from skimage.feature import blob_doh
#blobs_doh = blob_doh(image_gray, max_sigma=30, threshold=.01)

In [ ]:
zvals = scan_data['z'].unique()
zvals.sort()
fig, axes = plt.subplots(7, 7, facecolor='white', figsize=(20, 20), sharey=True, sharex=True)

e_by_layer = []
for i, plane in enumerate(zvals):
    row, col = i/7, i%7
    df = scan_data.query('z == {0}'.format(plane))
    e_by_layer.append(np.sum(df.e))
    e_cum = np.sum(e_by_layer)
    
    axes[row][col].scatter(df.x, df.y, c=df.e, s=40*df.e, marker='8')
    axes[row][col].set_xlim(10,60)
    axes[row][col].set_ylim(30,100)
    
    axes[row][col].text(33,74, r'$\sum E_{{z}}$ = {0:.2f}'.format(e_cum))
    axes[row][col].text(35,82, r'$E_{{z}}$ = {0:.2f}'.format(np.sum(df.e)))
    axes[row][col].text(35,90, 'z = {0:.1f}'.format(plane))

plt.show()

In [ ]:
e_by_layer = np.array(e_by_layer)
dz = zvals[1:] - zvals[:-1]
z1 = zvals[:-1] + dz/2.
de_dz = np.abs(e_by_layer[1:] - e_by_layer[:-1])/dz

dz2 = z1[1:] - z1[:-1]
z2 = z1[:-1] + dz2/2.
d2e_dz2 = np.abs(de_dz[1:] - de_dz[:-1])/dz2

fix, axes = plt.subplots(1, 3, figsize=(10, 6),  facecolor='white')
axes[0].plot(zvals, e_by_layer)
axes[1].plot(z1, de_dz)
axes[2].plot(z2, d2e_dz2)

plt.show()

In [ ]:
# test parametric hex grid
grid_size = ((0., 100.), (0., 100.))
edge_len = 1
hlines_x = np.array([[p, p+edge_len] for p in np.linspace(0, 100, 51)])
hlines_y = np.ones((hlines_x.size, 2))

#plt.plot(x=hlines_x, y=hlines_y)
plt.show()